In [1]:
require 'bio'

true

In [66]:
# Retrieve sequences.
fasta_file = Bio::FlatFile.auto("./data/TAIR10_cds_20101214_updated.fa")
fasta_file.each_entry do |x|
    p x.naseq.translate[1..-2]
    abort
    protein = x.naseq.translate
    puts protein.to_fasta(header= x.entry_id)
    abort
end

"VGGKKKTKICDKVSHEEDRISQLPEPLISEILFHLSTKDSVRTSALSTKWRYLWQSVPGLDLDPYASSNTNTIVSFVESFFDSHRDSWIRKLRLDLGYHHDKYDLMSWIDAATTRRIQHLDVHCFHDNKIPLSIYTCTTLVHLRLRWAVLTNPEFVSLPCLKIMHFENVSYPNETTLQKLISGSPVLEELILFSTMYPKGNVLQLRSDTLKRLDINEFIDVVIYAPLLQCLRAKMYSTKNFQIISSGFPAKLDIDFVNTGGRYQKKKVIEDILIDISRVRDLVISSNTWKEFFLYSKSRPLLQFRYISHLNARFYISDLEMLPTLLESCPKLESLILVMSSFNPS"


In [89]:
# Modify databases
fastaoutput = File.open('./data/ara.fa', 'w')
fasta_file = Bio::FlatFile.auto("./data/TAIR10_cds_20101214_updated.fa")
fasta_file.each_entry do |x|
    x.definition = x.entry_id
    fastaoutput.write(x.to_s)
end

In [ ]:
# Modify databases
fastaoutput = File.open('./data/ara.fa', 'w')
fasta_file = Bio::FlatFile.auto("./data/TAIR10_cds_20101214_updated.fa")
fasta_file.each_entry do |x|å
    x.definition = x.entry_id
    fastaoutput.write(x.to_s)
end

In [1]:
# Retrieve query sequences
require 'rest-client'
require 'bio'

# Auxiliary function to fetch a URL correctly.
# 
# @param url [String] The URL to search.
# @return [String] The response for the URL.
def fetch(url, headers = {accept: "*/*"}, user = "", pass="")
	response = RestClient::Request.execute({
	  method: :get,
	  url: url.to_s,
	  user: user,
	  password: pass,
	  headers: headers})
	return response
	
	rescue RestClient::ExceptionWithResponse => e
	  $stderr.puts e.inspect
	  response = false
	  return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
	rescue RestClient::Exception => e
	  $stderr.puts e.inspect
	  response = false
	  return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
	rescue Exception => e
	  $stderr.puts e.inspect
	  response = false
	  return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
end

  
genes = File.open('./ArabidopsisSubNetwork_GeneList.txt', 'r')
genearray = genes.read.split() # this will read each line into an array
genes.close
  
fastaoutput = File.open('./data/ara_genes.fa', 'w')

genearray.each do |geneid|
  
  url = "http://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=ensemblgenomesgene&format=fasta&id=#{geneid}"
  res = fetch(url)  # we really should check that the return value is valid, but... 
  fastafile = Bio::FlatFile.new(Bio::FastaFormat, StringIO.new(res.body))
  fastafile.each_entry do |fasta|
    naseq = fasta.to_biosequence
    fastaoutput.puts naseq.output_fasta(geneid)
  end
end
puts "DONE"

DONE


In [4]:
require 'bio'
require 'stringio'

# blast is installed globally, so you don't need to include the full path to blastn
factory = Bio::Blast.local('blastn', './databases/athaliana')   # wherever your database is....a BLAST "factory" 
#factory = Bio::Blast.local('tblastx', '/home/osboxes/Databases/EBOLA') # 
# Matches using proteins are more biologically meaningfull!! More AA (DNA is more redundant)
# blastn is the least accurate.


$stderr.puts "Searching ... "  # output to "standard error", this is to output comments to yourself, or error messages
                               # that are not intended to be part of the output of the software (pink background)

# executes homology search. Returns Bio::Blast::Report object.
report = factory.query(">myseq\nAGCCCCGTTTTTTGCCACTGGAATTAAGCCAATAACGTCAAGTGCTAGATCTAGTGAGTGGTAT") # this is a Bio::Blast::Report
abort
# This is important for the assignment.

#puts factory.output   # to see the blast report, verbatim
#puts report.hits
# Iterates over each hit
report.each do |hit|
  print "#{hit.hit_id} : evalue #{hit.evalue}\t#{hit.target_id} at "
  puts "#{hit.lap_at}"   # this tells you start and end of both the query and the hit sequences
  hit.each do |hsp|
    puts hsp.qseq  # this is the gapped Alignment as text of the query
    puts hsp.hseq  # this is the gapped Alignment as text of the hit
  end
end

# Its possible that the query matches different places of the database (it considers it two separate matches).
# high scoring segment pair (both sequence aligned) (hsp).
# In a report you have multiple hits, each hit has multiple hsp

Searching ... 


Errno::ENOENT: No such file or directory - blastall

In [3]:
factory

#<Bio::Blast:0x00000001321e8ba0 @program="blastn", @db="./databases/athaliana", @blastall="blastall", @matrix=nil, @filter=nil, @output="", @parser=nil, @format=7, @options=[], @server="local">